<a href="https://colab.research.google.com/github/marie-ganon/Projet-Python-S1/blob/main/Mod%C3%A9lisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from urllib import request
from wordcloud import WordCloud, STOPWORDS 
import csv
import bs4
from tqdm.notebook import trange, tqdm
import seaborn as sns
import numpy as np

In [24]:
 df = pd.read_csv('https://raw.githubusercontent.com/marie-ganon/Projet-Python-S1/main/base.csv')

In [ ]:
#test = pd.read_csv('https://raw.githubusercontent.com/marie-ganon/Projet-Python-S1/main/movies_metadata.csv.zip')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.



Avant de modéliser quoi que ce soit, nous allons supprimer les 'NaN' de la base de données.

In [76]:
df = df.dropna()

# Analyse de la matrice de corrélation de la base

Rappelons la matrice de corrélation de la base :

In [77]:
corr = df.corr()
corr.style.background_gradient(cmap = 'coolwarm')

,Unnamed: 0,budget,id,popularity,revenue,runtime,vote_average,vote_count,collection,score_1,production_companies_id,pc_id_count
Unnamed: 0,1.000000,-0.148571,0.623423,-0.175530,-0.110522,-0.171669,-0.185557,-0.113463,-0.080594,-0.110614,0.287707,-0.246866
budget,-0.148571,1.000000,-0.096657,0.436337,0.766920,0.155912,0.068225,0.671645,0.180931,0.629228,-0.135599,0.144304
id,0.623423,-0.096657,1.000000,-0.053992,-0.067341,-0.095947,-0.136979,-0.055209,-0.117688,-0.054086,0.330078,-0.181285
popularity,-0.175530,0.436337,-0.053992,1.000000,0.501218,0.124911,0.146682,0.552508,0.177025,0.540582,-0.146485,0.089818
revenue,-0.110522,0.766920,-0.067341,0.501218,1.000000,0.121972,0.091945,0.809743,0.235846,0.784853,-0.103861,0.128344
runtime,-0.171669,0.155912,-0.095947,0.124911,0.121972,1.000000,0.155065,0.130619,-0.016928,0.130466,-0.090927,0.041570
vote_average,-0.185557,0.068225,-0.136979,0.146682,0.091945,0.155065,1.000000,0.140098,0.023347,0.147204,-0.082563,0.001795
vote_count,-0.113463,0.671645,-0.055209,0.552508,0.809743,0.130619,0.140098,1.000000,0.209809,0.994286,-0.116741,0.108258
collection,-0.080594,0.180931,-0.117688,0.177025,0.235846,-0.016928,0.023347,0.209809,1.000000,0.194312,-0.099509,0.034864
score_1,-0.110614,0.629228,-0.054086,0.540582,0.784853,0.130466,0.147204,0.994286,0.194312,1.000000,-0.109567,0.101393


Par construction de *score_1*, il n'est pas choquant d'observer une forte corrélation avec *popularity*, *vote_count* et *vote_average*.

On constate que bien qu'il y ait peu de données qui puissent être exploitées pour le budget et le revenu d'un film, ces deux variables sont cependant corrélées avec *score_1*, qui permet de quantifier la qualité d'un film.

Par ailleurs, la compagnie de production et la durée du film ne semblent guère influencer sa qualité. 

# Régression linéaire de *score_1* sur *budget* et *revenue*

Tout d'abord, récupérons les données nécessaires dans le dataset.

In [78]:
X = pd.DataFrame(np.c_[df.budget, df.revenue], columns= ['budget', 'revenue'])
Y = df.score_1

Séparons ensuite nos deux ensembles de données en bases d'apprentissage et de test.

In [82]:
from sklearn.model_selection import train_test_split
 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(25756, 2)
(6440, 2)
(25756,)
(6440,)


On effectue alors la régression linéaire voulue.

In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
 
lmodellineaire = LinearRegression()
lmodellineaire.fit(X_train, Y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Evaluons les performances du modèle sur l'échantillon d'apprentissage et sur celui de test :

In [84]:
# Evaluation du training set
from sklearn.metrics import r2_score
y_train_predict = lmodellineaire.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
r2 = r2_score(Y_train, y_train_predict)
 
print("Performance du modèle sur la base d'apprentissage")
print('--------------------------------------')
print("L'erreur quadratique moyenne est {}".format(rmse))
print('Le score R² est {}'.format(r2))
print('\n')
 
# Evaluation du testing set
y_test_predict = lmodellineaire.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)
 
print('Performance du modèle sur la base de test')
print('--------------------------------------')
print("L'erreur quadratique moyenne est {}".format(rmse))
print('Le score R² est {}'.format(r2))

Performance du modèle sur la base d'apprentissage
--------------------------------------
L'erreur quadratique moyenne est 2671.0388755570357
Le score R² est 0.6120609574121265


Performance du modèle sur la base de test
--------------------------------------
L'erreur quadratique moyenne est 2167.897726148996
Le score R² est 0.648478800442039


Bien que le R² soit plutôt bon, ce modèle génère une erreur quadratique moyenne assez forte sur la valeur de *score_1* (de l'ordre de 2500). Cela est certainement dû à la dispersion de score_1 :

In [86]:
df.score_1.describe()

count     32196.00000
mean       1013.59686
std        4169.92930
min           0.00000
25%          29.00000
50%          91.20000
75%         362.60000
max      114007.50000
Name: score_1, dtype: float64

Etudions les perfomances de ce modèle sur un échantillon comprenant les valeurs de *score_1* allant du 1er au 3ème quartile :

In [89]:
first_quart = np.percentile(df.score_1, 25)
third_quart = np.percentile(df.score_1, 75)

In [101]:
# Filtration des données
df_quart = df[df.score_1 > first_quart]
df_quart = df[df.score_1 < third_quart]

# Création des variables nécessaires
X = pd.DataFrame(np.c_[df_quart.budget, df_quart.revenue], columns= ['budget', 'revenue'])
Y = df_quart.score_1
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5)

# Lancement de la régression linéaire
lmodellineaire = LinearRegression()
lmodellineaire.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [102]:
# Evaluation du training set
from sklearn.metrics import r2_score
y_train_predict = lmodellineaire.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
r2 = r2_score(Y_train, y_train_predict)
 
print("Performance du modèle sur la base d'apprentissage")
print('--------------------------------------')
print("L'erreur quadratique moyenne est {}".format(rmse))
print('Le score R² est {}'.format(r2))
print('\n')
 
# Evaluation du testing set
y_test_predict = lmodellineaire.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
r2 = r2_score(Y_test, y_test_predict)
 
print('Performance du modèle sur la base de test')
print('--------------------------------------')
print("L'erreur quadratique moyenne est {}".format(rmse))
print('Le score R² est {}'.format(r2))

Performance du modèle sur la base d'apprentissage
--------------------------------------
L'erreur quadratique moyenne est 83.63004147117628
Le score R² est 0.0595354749583098


Performance du modèle sur la base de test
--------------------------------------
L'erreur quadratique moyenne est 84.01766747313141
Le score R² est 0.027584725994881265


Par cette nouvelle régression, nous avons certes diminué drastiquement l'erreur quadratique moyenne, cependant le R² a chuté (de 65% à 3% sur l'échantillon test).